In [3]:
import numpy as np
import astropy.coordinates as coord
from astropy.cosmology import FlatLambdaCDM
from astropy.table import Table
import scipy.stats as stats
import scipy.integrate as integrate
import matplotlib.pyplot as plt

In [7]:
def create_mock(clusters, z, nct_params, d_ref, kt_scale, H0, intr_scatter, kt_bias, f_bias, k):
    
    '''
    Takes in: 
    DATA - arrays of cluster names, best-fit scaling factors, redshifts, nct fit parameters, reference
    cosmology, kT scaling factors
    MODEL PARAMETERS - floats H0, intrinsic scatter (encompasses fit scatter), kT bias, flux bias, overall fit bias
    
    Computes:
    For each cluster, the likelihood that the observed scaling factor corresponds to a cosmology with a Hubble 
    constant of H0 (w/ other given model parameters)
    
    Returns:
    Sum of the log-likelihood of all clusters
    '''
    
    s_true = []
    s_noise = []
    s_obs = []
    
    trial_cosmo = FlatLambdaCDM(H0=H0, Om0=0.3) # trial cosmology using input H0
    
    for i in range(0, 14):

        dz = coord.Distance(z=z[i], cosmology=trial_cosmo) # compute d(z) according to trial cosmology
        
        # Mean scaling factor, adjusted for the X-ray kT & flux calibration, as well as the overall fit bias
        # This should maybe be scaled by the overall blinding factor?
        s_mean = np.log(np.sqrt(d_ref[i]/dz) * (1 + kt_scale[i] * kt_bias)) + f_bias
        
        s_true.append(stats.norm.rvs(loc=s_mean, scale=intr_scatter))
        s_noise.append(stats.nct.rvs(nct_params[0][i], nct_params[1][i], loc=nct_params[2][i],
                                     scale=nct_params[3][i]))
        #scale=0.0001
        s_obs.append(s_true[i] + s_noise[i]) # + s_noise[i])
    
    df = []
    nc = []
    mu = []
    std = []
    for i in range(0, 14):
        df.append(nct_params[0][i])
        nc.append(nct_params[1][i])
        mu.append(nct_params[2][i])
        std.append(nct_params[3][i])

    t = Table((clusters, s_obs, df, nc, mu, std, z, d_ref, kt_scale), 
              names=('clusters', 's_obs', 'df', 'nc', 'mu', 'std', 'redshift', 'd_ref', 'kt_scale'))
    t.write('mock_data_60_0.1.dat', format='ascii', overwrite=True)
        
    return(s_obs)

In [8]:
data = Table.read('actual_data_for_likelihood_1000.dat', format='ascii')

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
d_ref = coord.Distance(z=data['redshift'], cosmology=cosmo)

for i in range(0, 1):
    create_mock(data['clusters'], data['redshift'], [data['df'], data['nc'], data['mu'], data['std']], 
               d_ref, data['kt_scale'], 60, 0.1, 0.09, 0.024, i)

In [861]:
# small intrinsic scatter 0.05